In [1]:
import pandas as pd
#import nltk 
import numpy as np
from nltk.corpus import stopwords
import re
from nltk.stem import WordNetLemmatizer
from IPython.display import display
import pyspark
sc = pyspark.SparkContext
from pyspark.sql import SparkSession
from pyspark import SQLContext
from pyspark.sql import Window as W
import pyspark.sql.functions as F
from pyspark.sql.dataframe import DataFrame
from pyspark.sql.types import *
from pyspark.sql import *
spark = SparkSession \
    .builder \
    .getOrCreate()

In [ ]:
import tastingmovies

In [36]:
rating = pd.read_csv('data/'+'rating'+'.csv',header=0, parse_dates=['timestamp'], dtype = {"userId" : "str","movieId" : "str"})
display(rating.head())

,userId,movieId,rating,timestamp
0,1,2,3.5,2005-04-02 23:53:47
1,1,29,3.5,2005-04-02 23:31:16
2,1,32,3.5,2005-04-02 23:33:39
3,1,47,3.5,2005-04-02 23:32:07
4,1,50,3.5,2005-04-02 23:29:40


In [2]:
tag = pd.read_csv('data/'+'tag'+'.csv',header=0, parse_dates=['timestamp'], dtype = {"userId" : "str","movieId" : "str"})
display(tag.head())

rating = pd.read_csv('data/'+'rating'+'.csv',header=0, parse_dates=['timestamp'], dtype = {"userId" : "str","movieId" : "str"})
display(rating.head())

#genome_scores = pd.read_csv('data/'+'genome_scores'+'.csv',header=0, dtype = {"tagId" : "str","movieId" : "str"})
#display(genome_scores.head())

genome_tags = pd.read_csv('data/'+'genome_tags'+'.csv',header=0, dtype = {"tagId" : "str","tag" : "str"})
display(genome_tags.head())

movie = pd.read_csv('data/'+'movie'+'.csv',header=0, dtype = {"title" : "str","movieId" : "str"})
display(movie.head())

,userId,movieId,tag,timestamp
0,18,4141,Mark Waters,2009-04-24 18:19:40
1,65,208,dark hero,2013-05-10 01:41:18
2,65,353,dark hero,2013-05-10 01:41:19
3,65,521,noir thriller,2013-05-10 01:39:43
4,65,592,dark hero,2013-05-10 01:41:18


,userId,movieId,rating,timestamp
0,1,2,3.5,2005-04-02 23:53:47
1,1,29,3.5,2005-04-02 23:31:16
2,1,32,3.5,2005-04-02 23:33:39
3,1,47,3.5,2005-04-02 23:32:07
4,1,50,3.5,2005-04-02 23:29:40


,tagId,tag
0,1,007
1,2,007 (series)
2,3,18th century
3,4,1920s
4,5,1930s


,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [127]:
genome_scores = pd.merge(genome_scores,genome_scores.groupby('movieId').mean().reset_index().rename(columns={'relevance':'avg_relevance'}),on=['movieId'],how='left')
genome_scores

,movieId,tagId,relevance,avg_relevance
0,1,1,0.02500,0.160223
1,1,2,0.02500,0.160223
2,1,3,0.05775,0.160223
3,1,4,0.09675,0.160223
4,1,5,0.14675,0.160223
...,...,...,...,...
11709763,131170,1124,0.58775,0.162884
11709764,131170,1125,0.01075,0.162884
11709765,131170,1126,0.01575,0.162884
11709766,131170,1127,0.11450,0.162884


In [133]:
genome_scores_most_relevant = genome_scores[genome_scores.relevance>=genome_scores.avg_relevance]
genome_scores_most_relevant

,movieId,tagId,relevance,avg_relevance
5,1,6,0.21700,0.160223
7,1,8,0.26275,0.160223
8,1,9,0.26200,0.160223
10,1,11,0.57700,0.160223
12,1,13,0.18800,0.160223
...,...,...,...,...
11709748,131170,1109,0.20875,0.162884
11709749,131170,1110,0.47125,0.162884
11709755,131170,1116,0.31125,0.162884
11709762,131170,1123,0.64375,0.162884


In [26]:
tag = pd.read_csv('data/'+'tag'+'.csv',header=0, parse_dates=['timestamp'], dtype = {"userId" : "str","movieId" : "str"})
display(tag.head())

,userId,movieId,tag,timestamp
0,18,4141,Mark Waters,2009-04-24 18:19:40
1,65,208,dark hero,2013-05-10 01:41:18
2,65,353,dark hero,2013-05-10 01:41:19
3,65,521,noir thriller,2013-05-10 01:39:43
4,65,592,dark hero,2013-05-10 01:41:18


In [134]:
BASE = "genome_scores_most_relevant"
genome_scores_most_relevant.to_pickle("data/" + BASE + ".pkl")

In [29]:
BASE = "genome_scores_most_relevant"
genome_scores_most_relevant = pd.read_pickle("data/" + BASE + ".pkl")
genome_tags_relevance=pd.merge(genome_scores_most_relevant,genome_tags,how='left',on=['tagId'])
genome_tags_relevance

,movieId,tagId,relevance,avg_relevance,tag
0,1,6,0.21700,0.160223,1950s
1,1,8,0.26275,0.160223,1970s
2,1,9,0.26200,0.160223,1980s
3,1,11,0.57700,0.160223,3d
4,1,13,0.18800,0.160223,80s
...,...,...,...,...,...
3571640,131170,1109,0.20875,0.162884,wilderness
3571641,131170,1110,0.47125,0.162884,wine
3571642,131170,1116,0.31125,0.162884,women
3571643,131170,1123,0.64375,0.162884,writers


In [30]:
BASE = "genome_tags_relevance"
genome_tags_relevance.to_pickle("data/" + BASE + ".pkl")

In [31]:
tag_relevance = pd.merge(tag,genome_tags_relevance,how='left',on=['tag','movieId'])
tag_relevance

,userId,movieId,tag,timestamp,tagId,relevance,avg_relevance
0,18,4141,Mark Waters,2009-04-24 18:19:40,NaN,NaN,NaN
1,65,208,dark hero,2013-05-10 01:41:18,288,0.94950,0.129743
2,65,353,dark hero,2013-05-10 01:41:19,288,0.99800,0.160859
3,65,521,noir thriller,2013-05-10 01:39:43,712,0.98900,0.133372
4,65,592,dark hero,2013-05-10 01:41:18,288,0.99775,0.145021
...,...,...,...,...,...,...,...
465559,138446,55999,dragged,2013-01-23 23:29:32,NaN,NaN,NaN
465560,138446,55999,Jason Bateman,2013-01-23 23:29:38,NaN,NaN,NaN
465561,138446,55999,quirky,2013-01-23 23:29:38,829,0.68150,0.091041
465562,138446,55999,sad,2013-01-23 23:29:32,871,0.21250,0.091041


In [35]:
tag_relevance

,userId,movieId,tag,timestamp,tagId,relevance,avg_relevance
0,18,4141,Mark Waters,2009-04-24 18:19:40,NaN,NaN,NaN
1,65,208,dark hero,2013-05-10 01:41:18,288,0.94950,0.129743
2,65,353,dark hero,2013-05-10 01:41:19,288,0.99800,0.160859
3,65,521,noir thriller,2013-05-10 01:39:43,712,0.98900,0.133372
4,65,592,dark hero,2013-05-10 01:41:18,288,0.99775,0.145021
...,...,...,...,...,...,...,...
465559,138446,55999,dragged,2013-01-23 23:29:32,NaN,NaN,NaN
465560,138446,55999,Jason Bateman,2013-01-23 23:29:38,NaN,NaN,NaN
465561,138446,55999,quirky,2013-01-23 23:29:38,829,0.68150,0.091041
465562,138446,55999,sad,2013-01-23 23:29:32,871,0.21250,0.091041


In [38]:
rating = rating \
                .rename(columns={'timestamp':'timestamp_rating', 'rating':'rating_usr'})


In [40]:
tags_rating00 = pd.merge(tag_relevance, rating, how="left", on=['userId','movieId']).sort_values(['timestamp','userId','movieId'],ascending=[1,0,0])

tags_rating = tags_rating00[tags_rating00.rating_usr.notnull()]

pre_tag=pd.merge(tags_rating,movie.drop(columns=['genres']),how='left',on=['movieId'])


'''
Rellenar los NA con el valor de 1% bajo la lógica que si no tiene un valor de relevancia asociado es 
porque no es tan relevante para esa película y, para no matar el valor por el que luego quisiéramos 
multiplicar a esa relevancia, no la volvemos 0 sino que la dejamos en un valor bajo como puede ser 1%
'''


pre_tag['tag_relevance_movie']  = pre_tag['relevance'].fillna(.01)#*pre_tag['rating_usr'] 

min_timestamps = pre_tag.groupby(['userId','movieId']).agg(
            {
                'timestamp':'min',
                'timestamp_rating':'min'
            }
        ).reset_index()

min_timestamps['timestamp_movie'] = min_timestamps[['timestamp','timestamp_rating']].min(axis=1)



tag = pd.merge(min_timestamps[['userId','movieId','timestamp_movie']],pre_tag,on=['userId','movieId'],how='left')


In [41]:
tag

,userId,movieId,timestamp_movie,tag,timestamp,tagId,relevance,avg_relevance,rating_usr,timestamp_rating,title,tag_relevance_movie
0,100031,1250,2006-10-04 02:12:54,war,2006-10-04 02:14:24,1096,0.95800,0.169697,5.0,2006-10-04 02:12:54,"Bridge on the River Kwai, The (1957)",0.95800
1,100074,1701,2012-01-17 02:46:38,Woody Allen,2012-01-17 03:04:49,NaN,NaN,NaN,5.0,2012-01-17 02:46:38,Deconstructing Harry (1997),0.01000
2,100074,1701,2012-01-17 02:46:38,hilarious,2012-01-17 03:04:56,505,0.56175,0.119156,5.0,2012-01-17 02:46:38,Deconstructing Harry (1997),0.56175
3,100074,2338,2012-01-17 03:12:52,white guy with Jamaican/Caribbean accent,2012-01-17 03:12:52,NaN,NaN,NaN,2.0,2012-01-17 03:13:14,I Still Know What You Did Last Summer (1998),0.01000
4,100074,2338,2012-01-17 03:12:52,Jack Black,2012-01-17 03:12:57,NaN,NaN,NaN,2.0,2012-01-17 03:13:14,I Still Know What You Did Last Summer (1998),0.01000
...,...,...,...,...,...,...,...,...,...,...,...,...
391440,99998,1921,2015-02-28 23:57:47,insanity,2015-02-28 23:58:18,547,0.98050,0.188004,4.5,2015-02-28 23:57:47,Pi (1998),0.98050
391441,99998,3504,2015-02-28 23:23:21,Sidney Lumet,2015-02-28 23:23:29,NaN,NaN,NaN,5.0,2015-02-28 23:23:21,Network (1976),0.01000
391442,99998,3730,2015-02-28 23:26:12,psychological,2015-02-28 23:26:33,823,0.93900,0.181968,4.0,2015-02-28 23:26:12,"Conversation, The (1974)",0.93900
391443,99998,3730,2015-02-28 23:26:12,character study,2015-02-28 23:26:37,194,0.98075,0.181968,4.0,2015-02-28 23:26:12,"Conversation, The (1974)",0.98075


In [45]:
tag['tag_clean'] = [preproc_text(x) for x in tag.tag]
tag['tags_list'] = tag.tag_clean.str.split(expand=False)

counts_tags = tag.tag_clean.str.split(expand=True).stack().value_counts()
df_counts_tags = counts_tags.to_frame()
df_counts_tags.columns = ['frec_tag']
df_counts_tags['word'] = df_counts_tags.index
df_counts_tags.reset_index(drop=True, inplace=True)
df_counts_tags['num_charac'] = [len(x) for x in df_counts_tags.word]

In [46]:
df_counts_tags01 = df_counts_tags[(df_counts_tags.num_charac>1) & (df_counts_tags.frec_tag>2000)]

def filter_tags(tagtmp):
    return([x for x in df_counts_tags01.word if x in tagtmp])

tag['tags_2consider'] = [filter_tags(a) for a in tag.tags_list]
tag['num_words'] = [len(x) for x in tag.tags_2consider]
#self.tag['timestamp_movie'] = self.tag[['timestamp','timestamp_rating']].min(axis=1)

tag_sub = tag[tag.num_words>0][['userId','movieId','timestamp_movie','tags_2consider','rating_usr','tag_relevance_movie']]
tag_sub['I'] = 1

tag_sub_movies = tag[tag.num_words>0][['userId','movieId','title','timestamp_movie','rating_usr','genres_list']]
tag_sub_movies['I'] = 1

displary(tag_sub)
display(tag_sub_movies)

KeyError: "['genres_list'] not in index"

In [13]:
del globals()['preproc_text']
import nltk
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
stop_words = set(stopwords.words("english")) 
stemmer = WordNetLemmatizer()
def preproc_text(text_tmp): #,documents)  


    #print('Remove all the special characters')
    document = re.sub(r'\W', ' ', str(text_tmp))

    #print('Remove all single characters')
    document = re.sub(r'\s+[a-zA-Z]\s+', ' ', document)

    #print('Remove single characters from the start')
    document = re.sub(r'\^[a-zA-Z]\s+', ' ', document) 

    #print('Substituting multiple spaces with single space')
    document = re.sub(r'\s+', ' ', document, flags=re.I)

    #print('Removing prefixed "b" ')
    document = re.sub(r'^b\s+', '', document)

    #print('Converting to Lowercase')
    document = document.lower()

    #print('Lemmatization')
    document = document.split()

    document = [stemmer.lemmatize(word) for word in document]
    document = [word for word in document if not word in stop_words]
    document = ' '.join(document)

    #documents.append(document)
    return(document)


[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/jalfredomb/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [14]:
genome_tags_relevance['tag_clean'] = [preproc_text(x) for x in genome_tags_relevance.tag]
genome_tags_relevance

,movieId,tagId,relevance,avg_relevance,tag,tag_clean
0,1,6,0.21700,0.160223,1950s,1950s
1,1,8,0.26275,0.160223,1970s,1970s
2,1,9,0.26200,0.160223,1980s,1980s
3,1,11,0.57700,0.160223,3d,3d
4,1,13,0.18800,0.160223,80s,80
...,...,...,...,...,...,...
3571640,131170,1109,0.20875,0.162884,wilderness,wilderness
3571641,131170,1110,0.47125,0.162884,wine,wine
3571642,131170,1116,0.31125,0.162884,women,woman
3571643,131170,1123,0.64375,0.162884,writers,writer


In [15]:
genome_tags_relevance['tags_list'] = genome_tags_relevance.tag_clean.str.split(expand=False)
genome_tags_relevance

,movieId,tagId,relevance,avg_relevance,tag,tag_clean,tags_list
0,1,6,0.21700,0.160223,1950s,1950s,[1950s]
1,1,8,0.26275,0.160223,1970s,1970s,[1970s]
2,1,9,0.26200,0.160223,1980s,1980s,[1980s]
3,1,11,0.57700,0.160223,3d,3d,[3d]
4,1,13,0.18800,0.160223,80s,80,[80]
...,...,...,...,...,...,...,...
3571640,131170,1109,0.20875,0.162884,wilderness,wilderness,[wilderness]
3571641,131170,1110,0.47125,0.162884,wine,wine,[wine]
3571642,131170,1116,0.31125,0.162884,women,woman,[woman]
3571643,131170,1123,0.64375,0.162884,writers,writer,[writer]


In [17]:
counts_tags_genome = genome_tags_relevance.tag_clean.str.split(expand=True).stack().value_counts()
df_counts_tags_genome = counts_tags_genome.to_frame()
df_counts_tags_genome

,0
good,93691
oscar,77229
best,59785
nudity,59519
movie,58368
...,...
beatles,27
hannibal,24
lecter,24
figure,22


In [18]:
df_counts_tags_genome.columns = ['frec_tag']
df_counts_tags_genome['word'] = df_counts_tags_genome.index
df_counts_tags_genome.reset_index(drop=True, inplace=True)
df_counts_tags_genome['num_charac'] = [len(x) for x in df_counts_tags_genome.word]
df_counts_tags_genome

,frec_tag,word,num_charac
0,93691,good,4
1,77229,oscar,5
2,59785,best,4
3,59519,nudity,6
4,58368,movie,5
...,...,...,...
1087,27,beatles,7
1088,24,hannibal,8
1089,24,lecter,6
1090,22,figure,6


In [19]:
df_counts_tags_genome01 = df_counts_tags_genome[(df_counts_tags_genome.num_charac>1)]

def filter_tags(tagtmp):
    return([x for x in df_counts_tags_genome01.word if x in tagtmp])

genome_tags_relevance['tags_2consider'] = [filter_tags(a) for a in genome_tags_relevance.tags_list]
genome_tags_relevance

,movieId,tagId,relevance,avg_relevance,tag,tag_clean,tags_list,tags_2consider
0,1,6,0.21700,0.160223,1950s,1950s,[1950s],[1950s]
1,1,8,0.26275,0.160223,1970s,1970s,[1970s],[1970s]
2,1,9,0.26200,0.160223,1980s,1980s,[1980s],[1980s]
3,1,11,0.57700,0.160223,3d,3d,[3d],[3d]
4,1,13,0.18800,0.160223,80s,80,[80],[80]
...,...,...,...,...,...,...,...,...
3571640,131170,1109,0.20875,0.162884,wilderness,wilderness,[wilderness],[wilderness]
3571641,131170,1110,0.47125,0.162884,wine,wine,[wine],[wine]
3571642,131170,1116,0.31125,0.162884,women,woman,[woman],[woman]
3571643,131170,1123,0.64375,0.162884,writers,writer,[writer],[writer]


In [20]:
genome_tags_relevance['num_words'] = [len(x) for x in genome_tags_relevance.tags_2consider]
        #self.tag['timestamp_movie'] = self.tag[['timestamp','timestamp_rating']].min(axis=1)

genome_tags_relevance_sub = genome_tags_relevance[genome_tags_relevance.num_words>0]#[['userId','movieId','timestamp_movie','tags_2consider','rating_usr','tag_relevance_movie']]
genome_tags_relevance_sub['I'] = 1

In [21]:
genome_tags_relevance_sub

,movieId,tagId,relevance,avg_relevance,tag,tag_clean,tags_list,tags_2consider,num_words,I
0,1,6,0.21700,0.160223,1950s,1950s,[1950s],[1950s],1,1
1,1,8,0.26275,0.160223,1970s,1970s,[1970s],[1970s],1,1
2,1,9,0.26200,0.160223,1980s,1980s,[1980s],[1980s],1,1
3,1,11,0.57700,0.160223,3d,3d,[3d],[3d],1,1
4,1,13,0.18800,0.160223,80s,80,[80],[80],1,1
...,...,...,...,...,...,...,...,...,...,...
3571640,131170,1109,0.20875,0.162884,wilderness,wilderness,[wilderness],[wilderness],1,1
3571641,131170,1110,0.47125,0.162884,wine,wine,[wine],[wine],1,1
3571642,131170,1116,0.31125,0.162884,women,woman,[woman],[woman],1,1
3571643,131170,1123,0.64375,0.162884,writers,writer,[writer],[writer],1,1


In [22]:
tag

,userId,movieId,tag,timestamp
0,18,4141,Mark Waters,2009-04-24 18:19:40
1,65,208,dark hero,2013-05-10 01:41:18
2,65,353,dark hero,2013-05-10 01:41:19
3,65,521,noir thriller,2013-05-10 01:39:43
4,65,592,dark hero,2013-05-10 01:41:18
...,...,...,...,...
465559,138446,55999,dragged,2013-01-23 23:29:32
465560,138446,55999,Jason Bateman,2013-01-23 23:29:38
465561,138446,55999,quirky,2013-01-23 23:29:38
465562,138446,55999,sad,2013-01-23 23:29:32


In [56]:
import pandas as pd
from IPython.display import HTML
import nltk 
import numpy as np
from nltk.corpus import stopwords
import re
from nltk.stem import WordNetLemmatizer
import pyspark
sc = pyspark.SparkContext
from pyspark.sql import SparkSession
from pyspark import SQLContext
from pyspark.sql import Window as W
import pyspark.sql.functions as F
from pyspark.sql.dataframe import DataFrame
from pyspark.sql.types import *
from pyspark.sql import *



class prepmovies05():

    tag_init = None
    minimun_wordfreq = None
    stemmer = None
    stop_words = None
    rating = None
    genome_scores = None
    genome_tags = None
    movie = None
    minimun_to_hr=None
    key_columns = None
    spark = None

    def __init__(self,tag_init,minimun_wordfreq,rating,genome_scores,genome_tags,movie,
        spark,
        minimun_to_hr = 4,stop_words=None,stemmer=None,key_columns = ['userId', 'movieId','title', 'timestamp_TAG', 'timestamp_rating','timestamp_movie', 'rating_usr','high_rating']):

        self.spark = spark
        self.tag_init = tag_init
        self.minimun_wordfreq = minimun_wordfreq
        self.minimun_to_hr = minimun_to_hr
        self.key_columns = key_columns

        self.rating = rating \
            .rename(columns={'timestamp':'timestamp_rating', 'rating':'rating_usr'})

        self.genome_tags = genome_tags
        self.genome_scores = genome_scores

        #movie['I']=1
        movie['genres_list'] = movie.genres.str.split('|',expand=False)
        self.movie = movie


        if(stop_words is None):
            nltk.download('wordnet')
            self.stop_words = set(stopwords.words("english")) 

        if stemmer  is None :
            self.stemmer = WordNetLemmatizer()

        pass



    def join_tags_and_genome_ratings(self):

        print('join_tags_and_genome_ratings')
        genome_tags_relevance=pd.merge(self.genome_tags,self.genome_scores,how='left',on=['tagId'])

        tag_relevance = pd.merge(self.tag_init,genome_tags_relevance,how='left',on=['tag','movieId'])

        '''
        Rellenar los NA con el valor de 1% bajo la lógica que si no tiene un valor de relevancia asociado es 
        porque no es tan relevante para esa película y, para no matar el valor por el que luego quisiéramos 
        multiplicar a esa relevancia, no la volvemos 0 sino que la dejamos en un valor bajo como puede ser 1%
        '''
        
        tag_relevance['relevance'] = tag_relevance.relevance.fillna(.01)


        tags_rating00 = pd.merge(tag_relevance, self.rating, how="left", on=['userId','movieId']).sort_values(['timestamp','userId','movieId'],ascending=[1,0,0])

        tags_rating = tags_rating00[tags_rating00.rating_usr.notnull()]

        pre_tag=pd.merge(tags_rating,self.movie.drop(columns=['genres']),how='left',on=['movieId'])

        pre_tag['tag_relevance_movie']  = pre_tag['relevance']#*pre_tag['rating_usr'] 

        min_timestamps = pre_tag.groupby(['userId','movieId']).agg(
                    {
                        'timestamp':'min',
                        'timestamp_rating':'min'
                    }
                ).reset_index()

        min_timestamps['timestamp_movie'] = min_timestamps[['timestamp','timestamp_rating']].min(axis=1)



        self.tag = pd.merge(min_timestamps[['userId','movieId','timestamp_movie']],pre_tag,on=['userId','movieId'],how='left')

        return(self.tag)

    def preproc_text(self,text_tmp): #,documents)  


        #print('Remove all the special characters')
        document = re.sub(r'\W', ' ', str(text_tmp))
        
        #print('Remove all single characters')
        document = re.sub(r'\s+[a-zA-Z]\s+', ' ', document)
        
        #print('Remove single characters from the start')
        document = re.sub(r'\^[a-zA-Z]\s+', ' ', document) 
        
        #print('Substituting multiple spaces with single space')
        document = re.sub(r'\s+', ' ', document, flags=re.I)
        
        #print('Removing prefixed "b" ')
        document = re.sub(r'^b\s+', '', document)
        
        #print('Converting to Lowercase')
        document = document.lower()
        
        #print('Lemmatization')
        document = document.split()
        
        document = [self.stemmer.lemmatize(word) for word in document]
        document = [word for word in document if not word in self.stop_words]
        document = ' '.join(document)
        
        #documents.append(document)
        return(document)


    def process_tags01(self):
        
        print('---Remove all the special characters')
        print('------Remove all single characters')
        print('---------Remove single characters from the start')
        print('------------Substituting multiple spaces with single space')
        print('---------------Removing prefixed "b" ')
        print('---------------------Converting to Lowercase')
        print('------------------------Lemmatization')

        self.tag['tag_clean'] = [self.preproc_text(x) for x in self.tag.tag]
        self.tag['tags_list'] = self.tag.tag_clean.str.split(expand=False)

        counts_tags = self.tag.tag_clean.str.split(expand=True).stack().value_counts()
        df_counts_tags = counts_tags.to_frame()
        df_counts_tags.columns = ['frec_tag']
        df_counts_tags['word'] = df_counts_tags.index
        df_counts_tags.reset_index(drop=True, inplace=True)
        df_counts_tags['num_charac'] = [len(x) for x in df_counts_tags.word]

        df_counts_tags01 = df_counts_tags[(df_counts_tags.num_charac>1) & (df_counts_tags.frec_tag>self.minimun_wordfreq)]

        def filter_tags(tagtmp):
            return([x for x in df_counts_tags01.word if x in tagtmp])

        self.tag['tags_2consider'] = [filter_tags(a) for a in self.tag.tags_list]
        self.tag['num_words'] = [len(x) for x in self.tag.tags_2consider]
        #self.tag['timestamp_movie'] = self.tag[['timestamp','timestamp_rating']].min(axis=1)

        tag_sub = self.tag[self.tag.num_words>0][['userId','movieId','timestamp_movie','tags_2consider','rating_usr','tag_relevance_movie']]
        tag_sub['I'] = 1

        tag_sub_movies = self.tag[self.tag.num_words>0][['userId','movieId','title','timestamp_movie','rating_usr','genres_list']]
        tag_sub_movies['I'] = 1

        self.tag_sub = tag_sub
        self.tag_sub_movies=tag_sub_movies


    def pivoting_tag_sub(self):

        print('pivoting table with tags + relevance')

        tag_sub_stacked = self.tag_sub.set_index(['I','tag_relevance_movie','userId','movieId','timestamp_movie','rating_usr']) \
                                .tags_2consider \
                                .apply(pd.Series) \
                                .stack() \
                                .reset_index() \
                                .rename(columns={0:'tags_2consider'}) 

        tag_sub_pivot = pd.pivot_table(tag_sub_stacked,index=['userId','movieId','timestamp_movie','rating_usr'],columns='tags_2consider',
                                                            values=['I','tag_relevance_movie'],
                                                            aggfunc={
                                                                    'I' : np.max,
                                                                    'tag_relevance_movie' : np.mean
                                                                    },
                                                            fill_value=0
                                                            ).reset_index() 
       # \.rename(columns={'timestamp':'timestamp_TAG'})

#        def reindexing(xtmp):
#            if xtmp[1] == '':
#                res = xtmp[0]
#            elif ((xtmp[1] != '') & (xtmp[0] == 'I')):
#                res = xtmp[1]
#            else:
#                res = xtmp[1] + '_rtng_x_rlvnc'
#            return(res)
#        #df.columns = ['_'.join(col) for col in df.columns]
        tag_sub_pivot.columns = [self.reindexing(a) for a in tag_sub_pivot.columns]

        self.tag_sub_pivot = tag_sub_pivot

        return(self.tag_sub_pivot)



    def pivoting_movies_genres(self):

        print('pivoting ratings + movies´s genres')

        movie_stacked = self.tag_sub_movies.set_index(['I','userId','movieId','title','timestamp_movie','rating_usr']) \
                                .genres_list \
                                .apply(pd.Series) \
                                .stack() \
                                .reset_index() \
                                .rename(columns={0:'genres_list'}) 

        movie_pivot01 = pd.pivot_table(movie_stacked,index=['userId','movieId','timestamp_movie','title'],columns='genres_list',
                                                            values=['I','rating_usr'],
                                                            aggfunc={
                                                                    'I' : np.max,
                                                                    'rating_usr' : np.mean
                                                                    },
                                                            fill_value=0
                                                            ).reset_index()

        #df.columns = ['_'.join(col) for col in df.columns]
        movie_pivot01.columns = [self.reindexing(a) for a in movie_pivot01.columns]

        self.movie_pivot01 = movie_pivot01

        return(self.movie_pivot01)

    def reindexing(self,xtmp):
        if xtmp[1] == '':
            res = xtmp[0]
        elif ((xtmp[1] != '') & (xtmp[0] == 'I')):
            res = xtmp[1]
        else:
            res = xtmp[1] + '_rtng'
        return(res)



    def cumsum_tags_by_user(self):


        self.pivoting_tag_sub()

        key_columns = ['userId', 'movieId','timestamp_movie']
        cols_rtng = [name for name in self.tag_sub_pivot.columns if '_rtng' in name]
        cols_nortng = [re.sub(r'_rtng', '', x) for x in cols_rtng]

        subset = self.tag_sub_pivot[key_columns+cols_rtng+cols_nortng].sort_values(by=['userId','timestamp_movie'])
        subset['I_for_id'] = 1
        gb_subset = subset.groupby(['userId'])
        subset['id_movie_user'] = gb_subset['I_for_id'].cumsum(axis=0)
        self.df_cumsum_tags_by_user= subset.drop(columns=['movieId','timestamp_movie']).groupby('userId').expanding().sum().drop(columns=['userId','id_movie_user']).rename(columns={'I_for_id':'id_movie_user'}).reset_index().drop(columns=['level_1'])
        

    def cumsum_genres_by_user(self):


        self.pivoting_movies_genres()

        key_columns = ['userId', 'movieId','timestamp_movie']
        cols_rtng = [name for name in self.movie_pivot01.columns if '_rtng' in name]
        cols_nortng = [re.sub(r'_rtng', '', x) for x in cols_rtng]

        subset = self.movie_pivot01[key_columns+cols_rtng+cols_nortng].sort_values(by=['userId','timestamp_movie'])
        subset['I_for_id'] = 1
        gb_subset = subset.groupby(['userId'])
        subset['id_movie_user'] = gb_subset['I_for_id'].cumsum(axis=0)
        self.df_cumsum_genres_by_user= subset.drop(columns=['movieId','timestamp_movie']).groupby('userId').expanding().sum().drop(columns=['userId','id_movie_user']).rename(columns={'I_for_id':'id_movie_user'}).reset_index().drop(columns=['level_1'])








#    def pivoting_and_BDt0(self):

        

        #spark = self.spark



        #print('merging pivots into BD')
        #BDt0 = pd.merge(self.tag_sub_pivot,self.movie_pivot01, on=['userId','movieId'],how='left')
        #BDt0['timestamp_movie'] = BDt0[['timestamp_TAG','timestamp_rating']].min(axis=1)
        #BDt0['high_rating'] = [1 if x >=self.minimun_to_hr else 0 for x in BDt0['rating_usr']]
        #notkey_columns = [x for x in BDt0.columns if x not in self.key_columns]
        #notkey_columns
 #       self.BDt0 = BDt0[self.key_columns+notkey_columns]


        
#        return(self.BDt0)





#        tag_sub_stacked01 = self.tag_sub.set_index(['I','userId','movieId','timestamp','rating']) \
#                        .tags_2consider \
#                        .apply(pd.Series) \
#                        .stack() \
#                        .reset_index(level=0) \
#                        .rename(columns={0:'tags_2consider'}) 

#        tag_sub_pivot01 = pd.pivot_table(tag_sub_stacked01,index=['userId','movieId','timestamp','rating'],columns='tags_2consider',values=['I'],aggfunc=np.sum).fillna(0).reset_index() \
#                        .rename(columns={'timestamp':'timestamp_TAG'})



In [54]:
pm0 = prepmovies04(tag_init=tag,minimun_wordfreq=2000,rating=rating,
                              genome_scores=genome_scores,genome_tags=genome_tags,movie=movie,
                              spark=spark,
                              minimun_to_hr=4)

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/jalfredomb/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [55]:
pm0.tag_sub_movies = pm.tag_sub_movies
pm0.tag_sub = pm.tag_sub

In [57]:
pm = tastingmovies.prepmovies(tag_init=tag,minimun_wordfreq=2000,rating=rating,
                              genome_scores=genome_scores,genome_tags=genome_tags,movie=movie,
                              spark=spark,
                              minimun_to_hr=4)

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/jalfredomb/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


## Step 1

- Joining:

<br>

    - tag data set with genome tags in order to know the tag's relevance for the movie.
    
    - The Missings are converted to 1% value under the logic that if a tag doesn't appear on the relevance data set it is because is not that relevant to that movie in particular and, in orde to not nullify the 'relevance' value with zero (we will use it to multiply in the future) we change it with a very low valua such as 1% (0.01)

<br>

- We only keep the rows that have NO Missings at the rating columns hence this is going to be used for the build the objetive variable

<br>

- We create a sore of baseline-'a priori' to the relevance multiplying it with the rating that the user gave to that movie. It's king a ponderated relevance

<br>

- Spliting genres column into single words lists

In [58]:
pm.join_tags_and_genome_ratings()

join_tags_and_genome_ratings


,userId,movieId,timestamp_movie,tag,timestamp,tagId,relevance,rating_usr,timestamp_rating,title,genres_list,tag_relevance_movie
0,100031,1250,2006-10-04 02:12:54,war,2006-10-04 02:14:24,1096,0.95800,5.0,2006-10-04 02:12:54,"Bridge on the River Kwai, The (1957)","[Adventure, Drama, War]",0.95800
1,100074,1701,2012-01-17 02:46:38,Woody Allen,2012-01-17 03:04:49,NaN,0.01000,5.0,2012-01-17 02:46:38,Deconstructing Harry (1997),"[Comedy, Drama]",0.01000
2,100074,1701,2012-01-17 02:46:38,hilarious,2012-01-17 03:04:56,505,0.56175,5.0,2012-01-17 02:46:38,Deconstructing Harry (1997),"[Comedy, Drama]",0.56175
3,100074,2338,2012-01-17 03:12:52,white guy with Jamaican/Caribbean accent,2012-01-17 03:12:52,NaN,0.01000,2.0,2012-01-17 03:13:14,I Still Know What You Did Last Summer (1998),"[Horror, Mystery, Thriller]",0.01000
4,100074,2338,2012-01-17 03:12:52,Jack Black,2012-01-17 03:12:57,NaN,0.01000,2.0,2012-01-17 03:13:14,I Still Know What You Did Last Summer (1998),"[Horror, Mystery, Thriller]",0.01000
...,...,...,...,...,...,...,...,...,...,...,...,...
391440,99998,1921,2015-02-28 23:57:47,insanity,2015-02-28 23:58:18,547,0.98050,4.5,2015-02-28 23:57:47,Pi (1998),"[Drama, Sci-Fi, Thriller]",0.98050
391441,99998,3504,2015-02-28 23:23:21,Sidney Lumet,2015-02-28 23:23:29,NaN,0.01000,5.0,2015-02-28 23:23:21,Network (1976),"[Comedy, Drama]",0.01000
391442,99998,3730,2015-02-28 23:26:12,psychological,2015-02-28 23:26:33,823,0.93900,4.0,2015-02-28 23:26:12,"Conversation, The (1974)","[Drama, Mystery]",0.93900
391443,99998,3730,2015-02-28 23:26:12,character study,2015-02-28 23:26:37,194,0.98075,4.0,2015-02-28 23:26:12,"Conversation, The (1974)","[Drama, Mystery]",0.98075


## Step 2

- Cleaning, standarizing and lemmatizating all tags

- Spliting tags into single words lists

- Counting the frecuency of each through all 'documents'

- Counting the number of characters of each resulting tag-word

- Filter:

    - Removing all tag-words which frequency is lower that the minimum stablished (in this case, 500)
    
    - Removing all tag-words that are only a letter / character
    
- Generating two data set:
    
    - One with all tag-words, the user's rating for the movie and the respective ponderated rating-relevance
    
    - The other one with the each movie's genres lists and the user's rating

In [59]:
#pm.minimun_wordfreq=2000
pm.process_tags01()

---Remove all the special characters
------Remove all single characters
---------Remove single characters from the start
------------Substituting multiple spaces with single space
---------------Removing prefixed "b" 
---------------------Converting to Lowercase
------------------------Lemmatization


In [60]:
pm.tag_sub

,userId,movieId,timestamp_movie,tags_2consider,rating_usr,tag_relevance_movie,I
0,100031,1250,2006-10-04 02:12:54,[war],5.0,0.95800,1
4,100074,2338,2012-01-17 03:12:52,[black],2.0,0.01000,1
12,100074,3545,2012-01-17 02:46:50,"[book, based]",4.5,0.36625,1
13,100074,3545,2012-01-17 02:46:50,"[best, oscar]",4.5,0.01000,1
14,100074,3545,2012-01-17 02:46:50,"[best, oscar]",4.5,0.01000,1
...,...,...,...,...,...,...,...
391414,99998,111,2015-02-28 23:20:26,[dark],5.0,0.94100,1
391417,99998,1196,2015-02-28 23:31:23,[fantasy],4.5,0.86350,1
391418,99998,1203,2015-02-28 23:21:55,[good],5.0,0.76550,1
391430,99998,1237,2015-02-28 23:24:02,[atmospheric],5.0,0.80300,1


In [61]:
pm.tag_sub_movies

,userId,movieId,title,timestamp_movie,rating_usr,genres_list,I
0,100031,1250,"Bridge on the River Kwai, The (1957)",2006-10-04 02:12:54,5.0,"[Adventure, Drama, War]",1
4,100074,2338,I Still Know What You Did Last Summer (1998),2012-01-17 03:12:52,2.0,"[Horror, Mystery, Thriller]",1
12,100074,3545,Cabaret (1972),2012-01-17 02:46:50,4.5,"[Drama, Musical]",1
13,100074,3545,Cabaret (1972),2012-01-17 02:46:50,4.5,"[Drama, Musical]",1
14,100074,3545,Cabaret (1972),2012-01-17 02:46:50,4.5,"[Drama, Musical]",1
...,...,...,...,...,...,...,...
391414,99998,111,Taxi Driver (1976),2015-02-28 23:20:26,5.0,"[Crime, Drama, Thriller]",1
391417,99998,1196,Star Wars: Episode V - The Empire Strikes Back...,2015-02-28 23:31:23,4.5,"[Action, Adventure, Sci-Fi]",1
391418,99998,1203,12 Angry Men (1957),2015-02-28 23:21:55,5.0,[Drama],1
391430,99998,1237,"Seventh Seal, The (Sjunde inseglet, Det) (1957)",2015-02-28 23:24:02,5.0,[Drama],1


## Step 3

- Pivoting data set in order to generate:
    
        - binary columns for the presence of tag-word and genre for each user's evaluation movie
        
        - interaction columns for the presence of tag-word/genre and the rating and the ponderated rating-relevance

In [62]:
pm.cumsum_tags_by_user()

pivoting table with tags + relevance


In [64]:
pm.cumsum_genres_by_user()

pivoting ratings + movies´s genres


In [115]:
display(pm.tag_sub)
display(pm.tag_sub_movies)

,userId,movieId,timestamp_movie,tags_2consider,rating_usr,tag_relevance_movie,I
0,100031,1250,2006-10-04 02:12:54,[war],5.0,0.95800,1
4,100074,2338,2012-01-17 03:12:52,[black],2.0,0.01000,1
12,100074,3545,2012-01-17 02:46:50,"[book, based]",4.5,0.36625,1
13,100074,3545,2012-01-17 02:46:50,"[best, oscar]",4.5,0.01000,1
14,100074,3545,2012-01-17 02:46:50,"[best, oscar]",4.5,0.01000,1
...,...,...,...,...,...,...,...
391414,99998,111,2015-02-28 23:20:26,[dark],5.0,0.94100,1
391417,99998,1196,2015-02-28 23:31:23,[fantasy],4.5,0.86350,1
391418,99998,1203,2015-02-28 23:21:55,[good],5.0,0.76550,1
391430,99998,1237,2015-02-28 23:24:02,[atmospheric],5.0,0.80300,1


,userId,movieId,title,timestamp_movie,rating_usr,genres_list,I
0,100031,1250,"Bridge on the River Kwai, The (1957)",2006-10-04 02:12:54,5.0,"[Adventure, Drama, War]",1
4,100074,2338,I Still Know What You Did Last Summer (1998),2012-01-17 03:12:52,2.0,"[Horror, Mystery, Thriller]",1
12,100074,3545,Cabaret (1972),2012-01-17 02:46:50,4.5,"[Drama, Musical]",1
13,100074,3545,Cabaret (1972),2012-01-17 02:46:50,4.5,"[Drama, Musical]",1
14,100074,3545,Cabaret (1972),2012-01-17 02:46:50,4.5,"[Drama, Musical]",1
...,...,...,...,...,...,...,...
391414,99998,111,Taxi Driver (1976),2015-02-28 23:20:26,5.0,"[Crime, Drama, Thriller]",1
391417,99998,1196,Star Wars: Episode V - The Empire Strikes Back...,2015-02-28 23:31:23,4.5,"[Action, Adventure, Sci-Fi]",1
391418,99998,1203,12 Angry Men (1957),2015-02-28 23:21:55,5.0,[Drama],1
391430,99998,1237,"Seventh Seal, The (Sjunde inseglet, Det) (1957)",2015-02-28 23:24:02,5.0,[Drama],1


In [66]:
display(pm.tag_sub_pivot)
display(pm.movie_pivot01)

,userId,movieId,timestamp_movie,rating_usr,action,atmospheric,bad,based,best,black,...,plot_rtng,quirky_rtng,romance_rtng,sci_rtng,story_rtng,surreal_rtng,time_rtng,true_rtng,twist_rtng,war_rtng
0,100031,1250,2006-10-04 02:12:54,5.0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0000,0.0,0.0,0.0,0.958
1,100074,2338,2012-01-17 03:12:52,2.0,0,0,0,0,0,1,...,0.0,0.0,0.0,0.0,0.0,0.0000,0.0,0.0,0.0,0.000
2,100074,3545,2012-01-17 02:46:50,4.5,0,0,0,1,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0000,0.0,0.0,0.0,0.000
3,100074,4993,2012-01-17 03:18:51,4.0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0000,0.0,0.0,0.0,0.000
4,100074,7153,2012-01-17 03:19:41,4.5,0,1,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0000,0.0,0.0,0.0,0.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50163,99998,111,2015-02-28 23:20:26,5.0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0000,0.0,0.0,0.0,0.000
50164,99998,1196,2015-02-28 23:31:23,4.5,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0000,0.0,0.0,0.0,0.000
50165,99998,1203,2015-02-28 23:21:55,5.0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0000,0.0,0.0,0.0,0.000
50166,99998,1237,2015-02-28 23:24:02,5.0,0,1,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0000,0.0,0.0,0.0,0.000


,userId,movieId,timestamp_movie,title,(no genres listed),Action,Adventure,Animation,Children,Comedy,...,Film-Noir_rtng,Horror_rtng,IMAX_rtng,Musical_rtng,Mystery_rtng,Romance_rtng,Sci-Fi_rtng,Thriller_rtng,War_rtng,Western_rtng
0,100031,1250,2006-10-04 02:12:54,"Bridge on the River Kwai, The (1957)",0,0,1,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0
1,100074,2338,2012-01-17 03:12:52,I Still Know What You Did Last Summer (1998),0,0,0,0,0,0,...,0.0,2.0,0.0,0.0,2.0,0.0,0.0,2.0,0.0,0.0
2,100074,3545,2012-01-17 02:46:50,Cabaret (1972),0,0,0,0,0,0,...,0.0,0.0,0.0,4.5,0.0,0.0,0.0,0.0,0.0,0.0
3,100074,4993,2012-01-17 03:18:51,"Lord of the Rings: The Fellowship of the Ring,...",0,0,1,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,100074,7153,2012-01-17 03:19:41,"Lord of the Rings: The Return of the King, The...",0,1,1,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50163,99998,111,2015-02-28 23:20:26,Taxi Driver (1976),0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0
50164,99998,1196,2015-02-28 23:31:23,Star Wars: Episode V - The Empire Strikes Back...,0,1,1,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,4.5,0.0,0.0,0.0
50165,99998,1203,2015-02-28 23:21:55,12 Angry Men (1957),0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
50166,99998,1237,2015-02-28 23:24:02,"Seventh Seal, The (Sjunde inseglet, Det) (1957)",0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [65]:
display(pm.df_cumsum_genres_by_user)
display(pm.df_cumsum_tags_by_user)

,userId,(no genres listed)_rtng,Action_rtng,Adventure_rtng,Animation_rtng,Children_rtng,Comedy_rtng,Crime_rtng,Documentary_rtng,Drama_rtng,...,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western,id_movie_user
0,100031,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,5.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
1,100074,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.5,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,100074,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,4.5,...,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0
3,100074,0.0,0.0,4.0,0.0,0.0,0.0,0.0,3.5,4.5,...,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0
4,100074,0.0,0.0,4.0,0.0,0.0,0.0,0.0,3.5,4.5,...,1.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50163,99998,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,5.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
50164,99998,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,10.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,2.0
50165,99998,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,15.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,3.0
50166,99998,0.0,4.5,4.5,0.0,0.0,0.0,5.0,0.0,15.0,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,4.0


,userId,action_rtng,atmospheric_rtng,bad_rtng,based_rtng,best_rtng,black_rtng,book_rtng,classic_rtng,comedy_rtng,...,quirky,romance,sci,story,surreal,time,true,twist,war,id_movie_user
0,100031,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
1,100074,0.0,0.000000e+00,0.000000e+00,3.662500e-01,0.01,0.000000e+00,3.662500e-01,3.350000e-01,0.000000e+00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,100074,0.0,0.000000e+00,0.000000e+00,3.662500e-01,0.01,0.000000e+00,3.662500e-01,3.350000e-01,0.000000e+00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0
3,100074,0.0,0.000000e+00,0.000000e+00,3.662500e-01,0.01,0.000000e+00,3.662500e-01,3.350000e-01,1.357500e-01,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0
4,100074,0.0,0.000000e+00,0.000000e+00,3.662500e-01,0.01,1.000000e-02,3.662500e-01,3.350000e-01,1.357500e-01,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50163,99998,0.0,-2.220446e-16,2.220446e-16,6.661338e-16,0.00,-1.110223e-16,2.220446e-16,2.220446e-16,2.220446e-16,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
50164,99998,0.0,-2.220446e-16,2.220446e-16,6.661338e-16,0.00,-1.110223e-16,2.220446e-16,2.220446e-16,2.220446e-16,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0
50165,99998,0.0,8.030000e-01,2.220446e-16,6.661338e-16,0.00,-1.110223e-16,2.220446e-16,2.220446e-16,2.220446e-16,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0
50166,99998,0.0,8.030000e-01,2.220446e-16,6.661338e-16,0.00,-1.110223e-16,2.220446e-16,2.220446e-16,2.220446e-16,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0


In [67]:
BASE = "tag_sub_pivot"
pm.tag_sub_pivot.to_pickle("data/" + BASE + ".pkl")
BASE = "movie_pivot01"
pm.movie_pivot01.to_pickle("data/" + BASE + ".pkl")

In [116]:
BASE = "tag_sub"
pm.tag_sub.to_pickle("data/" + BASE + ".pkl")
BASE = "tag_sub_movies"
pm.tag_sub_movies.to_pickle("data/" + BASE + ".pkl")

In [117]:
BASE = "df_cumsum_tags_by_user"
pm.df_cumsum_tags_by_user.to_pickle("data/" + BASE + ".pkl")
BASE = "df_cumsum_genres_by_user"
pm.df_cumsum_genres_by_user.to_pickle("data/" + BASE + ".pkl")

In [70]:
pm.df_cumsum_tags_by_user_sp = spark.createDataFrame(pm.df_cumsum_tags_by_user)
pm.df_cumsum_tags_by_user_sp.toPandas()

,userId,action_rtng,atmospheric_rtng,bad_rtng,based_rtng,best_rtng,black_rtng,book_rtng,classic_rtng,comedy_rtng,...,quirky,romance,sci,story,surreal,time,true,twist,war,id_movie_user
0,100031,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
1,100074,0.0,0.000000e+00,0.000000e+00,3.662500e-01,0.01,0.000000e+00,3.662500e-01,3.350000e-01,0.000000e+00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,100074,0.0,0.000000e+00,0.000000e+00,3.662500e-01,0.01,0.000000e+00,3.662500e-01,3.350000e-01,0.000000e+00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0
3,100074,0.0,0.000000e+00,0.000000e+00,3.662500e-01,0.01,0.000000e+00,3.662500e-01,3.350000e-01,1.357500e-01,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0
4,100074,0.0,0.000000e+00,0.000000e+00,3.662500e-01,0.01,1.000000e-02,3.662500e-01,3.350000e-01,1.357500e-01,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50163,99998,0.0,-2.220446e-16,2.220446e-16,6.661338e-16,0.00,-1.110223e-16,2.220446e-16,2.220446e-16,2.220446e-16,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
50164,99998,0.0,-2.220446e-16,2.220446e-16,6.661338e-16,0.00,-1.110223e-16,2.220446e-16,2.220446e-16,2.220446e-16,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0
50165,99998,0.0,8.030000e-01,2.220446e-16,6.661338e-16,0.00,-1.110223e-16,2.220446e-16,2.220446e-16,2.220446e-16,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0
50166,99998,0.0,8.030000e-01,2.220446e-16,6.661338e-16,0.00,-1.110223e-16,2.220446e-16,2.220446e-16,2.220446e-16,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0


In [104]:
#pm.df_cumsum_genres_by_user_sp = spark.createDataFrame(pm.df_cumsum_genres_by_user)

cols_tags=pm.df_cumsum_tags_by_user_sp.columns
cols_genres=pm.df_cumsum_genres_by_user_sp.columns
cols_genres_in_tags=[x for x in cols_genres if x.lower() in cols_tags]
        
pm.df_cumsum_genres_by_user_sp = pm.df_cumsum_genres_by_user_sp \
            .select('*',*[F.col(a).alias(a+ '_genres') for a in cols_genres_in_tags]) \
            .drop(*cols_genres_in_tags)

pm.df_cumsum_genres_by_user_sp.toPandas()

,(no genres listed)_rtng,Adventure_rtng,Animation_rtng,Children_rtng,Crime_rtng,Documentary_rtng,Drama_rtng,Film-Noir_rtng,Horror_rtng,IMAX_rtng,...,Action_rtng_genres,Comedy_rtng_genres,Fantasy_rtng_genres,Romance_rtng_genres,War_rtng_genres,Action_genres,Comedy_genres,Fantasy_genres,Romance_genres,War_genres
0,0.0,5.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,1.0
1,0.0,0.0,0.0,0.0,0.0,0.0,4.5,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,4.0,0.0,0.0,0.0,0.0,4.5,0.0,0.0,4.0,...,0.0,0.0,4.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,0.0,4.0,0.0,0.0,0.0,3.5,4.5,0.0,0.0,4.0,...,0.0,0.0,4.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,0.0,4.0,0.0,0.0,0.0,3.5,4.5,0.0,2.0,4.0,...,0.0,0.0,4.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50163,0.0,0.0,0.0,0.0,5.0,0.0,5.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
50164,0.0,0.0,0.0,0.0,5.0,0.0,10.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
50165,0.0,0.0,0.0,0.0,5.0,0.0,15.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
50166,0.0,4.5,0.0,0.0,5.0,0.0,15.0,0.0,0.0,0.0,...,4.5,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [105]:
sorted(pm.df_cumsum_genres_by_user_sp.columns)

['(no genres listed)',
 '(no genres listed)_rtng',
 'Action_genres',
 'Action_rtng_genres',
 'Adventure',
 'Adventure_rtng',
 'Animation',
 'Animation_rtng',
 'Children',
 'Children_rtng',
 'Comedy_genres',
 'Comedy_rtng_genres',
 'Crime',
 'Crime_rtng',
 'Documentary',
 'Documentary_rtng',
 'Drama',
 'Drama_rtng',
 'Fantasy_genres',
 'Fantasy_rtng_genres',
 'Film-Noir',
 'Film-Noir_rtng',
 'Horror',
 'Horror_rtng',
 'IMAX',
 'IMAX_rtng',
 'Musical',
 'Musical_rtng',
 'Mystery',
 'Mystery_rtng',
 'Romance_genres',
 'Romance_rtng_genres',
 'Sci-Fi',
 'Sci-Fi_rtng',
 'Thriller',
 'Thriller_rtng',
 'War_genres',
 'War_rtng_genres',
 'Western',
 'Western_rtng',
 'id_movie_user_genres',
 'userId_genres']

In [106]:
cols_genres_in_tags

['Action_rtng',
 'Comedy_rtng',
 'Fantasy_rtng',
 'Romance_rtng',
 'War_rtng',
 'Action',
 'Comedy',
 'Fantasy',
 'Romance',
 'War']

In [107]:
df_cumsum_tags_genres_by_user_sp = pm.df_cumsum_tags_by_user_sp \
                                    .join(
                                            pm.df_cumsum_genres_by_user_sp,
                                            (
                                                (pm.df_cumsum_tags_by_user_sp.userId==pm.df_cumsum_genres_by_user_sp.userId_genres) 
                                                & (pm.df_cumsum_tags_by_user_sp.id_movie_user==pm.df_cumsum_genres_by_user_sp.id_movie_user_genres)
                                            )) \
                                    .drop(*['userId_genres','id_movie_user_genres'])
df_cumsum_tags_genres_by_user_sp.toPandas()

,userId,action_rtng,atmospheric_rtng,bad_rtng,based_rtng,best_rtng,black_rtng,book_rtng,classic_rtng,comedy_rtng,...,Action_rtng_genres,Comedy_rtng_genres,Fantasy_rtng_genres,Romance_rtng_genres,War_rtng_genres,Action_genres,Comedy_genres,Fantasy_genres,Romance_genres,War_genres
0,101214,0.000000e+00,7.215000e-01,0.000000e+00,0.000000e+00,0.01,0.000000e+00,0.000000e+00,0.000000e+00,7.575000e-01,...,5.0,5.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0
1,101767,0.000000e+00,7.175000e-01,0.000000e+00,0.000000e+00,0.00,6.472500e-01,0.000000e+00,0.000000e+00,1.914750e+00,...,0.0,4.5,4.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
2,104508,0.000000e+00,0.000000e+00,0.000000e+00,6.952500e-01,0.00,0.000000e+00,6.952500e-01,0.000000e+00,0.000000e+00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,105216,0.000000e+00,0.000000e+00,0.000000e+00,1.110223e-16,0.00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,105873,0.000000e+00,0.000000e+00,0.000000e+00,2.220446e-16,0.00,0.000000e+00,0.000000e+00,0.000000e+00,7.165000e-01,...,5.0,5.0,0.0,5.0,0.0,1.0,1.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50163,96370,1.000000e-02,-2.220446e-16,2.220446e-16,4.000000e-02,0.01,2.805750e+00,8.815000e-01,4.440892e-16,9.797500e-01,...,23.5,64.5,8.0,47.0,39.0,7.0,18.0,2.0,14.0,11.0
50164,97856,-1.110223e-16,-2.220446e-16,2.220446e-16,6.661338e-16,0.00,-1.110223e-16,2.220446e-16,2.220446e-16,9.395000e-01,...,3.5,3.5,3.5,0.0,0.0,1.0,1.0,1.0,0.0,0.0
50165,9815,9.776250e+00,2.083475e+01,1.000000e-02,2.591146e+01,0.01,1.116175e+01,2.200046e+01,1.090500e+01,2.459958e+01,...,60.0,165.5,54.0,97.0,0.0,13.0,39.0,12.0,22.0,0.0
50166,9815,1.351575e+01,3.406900e+01,1.000000e-02,3.329871e+01,0.02,1.633675e+01,2.875863e+01,2.005100e+01,2.796058e+01,...,98.5,215.0,74.5,140.5,0.0,23.0,51.0,17.0,32.0,0.0


In [109]:
pm.cols_rtng = [name for name in df_cumsum_tags_genres_by_user_sp.columns if '_rtng' in name]
pm.cols_nortng = [re.sub(r'_rtng', '', x) for x in pm.cols_rtng]


display(pm.cols_rtng)
display(pm.cols_nortng)

['action_rtng',
 'atmospheric_rtng',
 'bad_rtng',
 'based_rtng',
 'best_rtng',
 'black_rtng',
 'book_rtng',
 'classic_rtng',
 'comedy_rtng',
 'dark_rtng',
 'dvd_rtng',
 'ending_rtng',
 'fantasy_rtng',
 'fi_rtng',
 'film_rtng',
 'funny_rtng',
 'good_rtng',
 'great_rtng',
 'movie_rtng',
 'nudity_rtng',
 'oscar_rtng',
 'plot_rtng',
 'quirky_rtng',
 'romance_rtng',
 'sci_rtng',
 'story_rtng',
 'surreal_rtng',
 'time_rtng',
 'true_rtng',
 'twist_rtng',
 'war_rtng',
 '(no genres listed)_rtng',
 'Adventure_rtng',
 'Animation_rtng',
 'Children_rtng',
 'Crime_rtng',
 'Documentary_rtng',
 'Drama_rtng',
 'Film-Noir_rtng',
 'Horror_rtng',
 'IMAX_rtng',
 'Musical_rtng',
 'Mystery_rtng',
 'Sci-Fi_rtng',
 'Thriller_rtng',
 'Western_rtng',
 'Action_rtng_genres',
 'Comedy_rtng_genres',
 'Fantasy_rtng_genres',
 'Romance_rtng_genres',
 'War_rtng_genres']

['action',
 'atmospheric',
 'bad',
 'based',
 'best',
 'black',
 'book',
 'classic',
 'comedy',
 'dark',
 'dvd',
 'ending',
 'fantasy',
 'fi',
 'film',
 'funny',
 'good',
 'great',
 'movie',
 'nudity',
 'oscar',
 'plot',
 'quirky',
 'romance',
 'sci',
 'story',
 'surreal',
 'time',
 'true',
 'twist',
 'war',
 '(no genres listed)',
 'Adventure',
 'Animation',
 'Children',
 'Crime',
 'Documentary',
 'Drama',
 'Film-Noir',
 'Horror',
 'IMAX',
 'Musical',
 'Mystery',
 'Sci-Fi',
 'Thriller',
 'Western',
 'Action_genres',
 'Comedy_genres',
 'Fantasy_genres',
 'Romance_genres',
 'War_genres']

In [110]:
list_rtngacum = [F.round(F.col(x+'_rtng')/F.col(x),2).alias(x+'_avg_rtng_acum') for x in cols_nortng]
list_rtngacum

[Column<'round((action_rtng / action), 2) AS `action_avg_rtng_acum`'>,
 Column<'round((atmospheric_rtng / atmospheric), 2) AS `atmospheric_avg_rtng_acum`'>,
 Column<'round((bad_rtng / bad), 2) AS `bad_avg_rtng_acum`'>,
 Column<'round((based_rtng / based), 2) AS `based_avg_rtng_acum`'>,
 Column<'round((best_rtng / best), 2) AS `best_avg_rtng_acum`'>,
 Column<'round((black_rtng / black), 2) AS `black_avg_rtng_acum`'>,
 Column<'round((book_rtng / book), 2) AS `book_avg_rtng_acum`'>,
 Column<'round((classic_rtng / classic), 2) AS `classic_avg_rtng_acum`'>,
 Column<'round((comedy_rtng / comedy), 2) AS `comedy_avg_rtng_acum`'>,
 Column<'round((dark_rtng / dark), 2) AS `dark_avg_rtng_acum`'>,
 Column<'round((dvd_rtng / dvd), 2) AS `dvd_avg_rtng_acum`'>,
 Column<'round((ending_rtng / ending), 2) AS `ending_avg_rtng_acum`'>,
 Column<'round((fantasy_rtng / fantasy), 2) AS `fantasy_avg_rtng_acum`'>,
 Column<'round((fi_rtng / fi), 2) AS `fi_avg_rtng_acum`'>,
 Column<'round((film_rtng / film), 2) A

In [113]:
df_avgrtng_acum_by_user_sp = df_cumsum_tags_genres_by_user_sp \
                .select('userId','id_movie_user',*list_rtngacum)
pm.df_avgrtng_acum_by_user_pd = df_avgrtng_acum_by_user_sp.toPandas()
pm.df_avgrtng_acum_by_user_pd

,userId,id_movie_user,action_avg_rtng_acum,atmospheric_avg_rtng_acum,bad_avg_rtng_acum,based_avg_rtng_acum,best_avg_rtng_acum,black_avg_rtng_acum,book_avg_rtng_acum,classic_avg_rtng_acum,...,plot_avg_rtng_acum,quirky_avg_rtng_acum,romance_avg_rtng_acum,sci_avg_rtng_acum,story_avg_rtng_acum,surreal_avg_rtng_acum,time_avg_rtng_acum,true_avg_rtng_acum,twist_avg_rtng_acum,war_avg_rtng_acum
0,101214,2.0,NaN,0.72,NaN,NaN,0.01,NaN,NaN,NaN,...,NaN,NaN,NaN,0.97,NaN,0.71,NaN,NaN,NaN,NaN
1,101767,4.0,NaN,0.72,NaN,NaN,NaN,0.65,NaN,NaN,...,0.99,0.71,NaN,NaN,NaN,0.98,NaN,NaN,0.97,NaN
2,104508,1.0,NaN,NaN,NaN,0.70,NaN,NaN,0.70,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.74,NaN
3,105216,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,105873,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0.91,NaN,NaN,0.99,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50163,96370,63.0,0.01,NaN,NaN,0.01,0.01,0.94,0.88,NaN,...,NaN,NaN,0.50,NaN,0.98,NaN,0.92,0.49,0.86,0.01
50164,97856,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0.98,NaN,NaN,NaN,NaN
50165,9815,105.0,0.75,0.72,0.01,0.72,0.01,0.66,0.76,0.64,...,0.89,0.78,0.60,0.78,0.61,0.78,0.72,0.71,0.81,NaN
50166,9815,170.0,0.71,0.68,0.01,0.74,0.01,0.65,0.78,0.57,...,0.80,0.77,0.57,0.79,0.59,0.82,0.76,0.70,0.81,0.58


In [114]:
BASE = "df_avgrtng_acum_by_user_pd"
pm.df_avgrtng_acum_by_user_pd.to_pickle("data/" + BASE + ".pkl")